In [7]:
import os
import numpy as np
import pandas as pd
import math
from sklearn import metrics
# from imblearn.metrics import specificity_score
# from utils import makedir
import re
from sklearn.preprocessing import MinMaxScaler

#Model 
import torch
import torch.nn as nn                     # general structure of a net 
from torch.nn.parameter import Parameter  # for weights and bias
import torch.nn.functional as F           # for Relu
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader # to data loader
#from torch.utils import data
import torch.optim as optim

import json                              # import data from json file
import argparse
import ipdb
from src.methods.brits.lightningmodule import BRITSLightningModule
from src.methods.brits.modules import MultiTaskBRITS
from multitask_missing_q10_ours import Model_brits_att

 # Hyperparameters 
SEQ_LEN = 40                           # number of period in the ts, t = 1, 2, 3, 4, 5.
RNN_HID_SIZE = 32                     # hidden node of the rnn 
batch_size = 32
model_name = 'BRITS_ATT' # RITS
question = 'feeling_lately'
open_face = 'eye_gaze'
epochs = 100
#N_SERIES = 12                          # number of series Rd, 12:eye, 136:landmark,14:action unit
lr = 1e-3
repetitions = 5
ratio_missing = 0.05
type_missing = 'Random' # CMV
rnn_name = 'GRU' # GRU
experiment_name = '777'

if open_face=='action_unit':
    N_SERIES = 14  # 14, for action unit
elif open_face=='eye_gaze':
    N_SERIES = 12
elif open_face=='landmark':
    N_SERIES = 136
elif open_face=='all':
    N_SERIES = (14+12+136)

In [4]:
from src.datamodules.daicwoz import DAICWOZDatamodule

dm = DAICWOZDatamodule(
    label='phq8',
    question=question,
    open_face=open_face,
    delta_steps=1,
    delta_average=False,
    regen=True,
    ratio_missing=ratio_missing,
    type_missing=type_missing,
    batch_size=batch_size,
    num_workers=1,
    ricardo=False,
    val_ratio=0.2
)
dm.prepare_data()
dm.setup()
dm_train = dm.train_dataloader()
dm_val = dm.val_dataloader()

In [8]:
model_ric = Model_brits_att(rnn_name=rnn_name)
optimizer_ric = optim.Adam(model_ric.parameters(), lr = lr)

NameError: name 'N_SERIES' is not defined

In [ ]:
for epoch in range(epochs):
    model_ric.train()

    run_loss = 0.0

    for idx, (data, batch) in enumerate(zip(dm_train, data_iter)):
        params_type = next(iter(model_ric.parameters()))

        batch['forward']['values'] = data['X'].type_as(params_type)
        batch['forward']['masks'] = data['missing_mask'].type_as(params_type)
        batch['forward']['deltas'] = data['deltas'].type_as(params_type)
        batch['forward']['evals'] = data['X_ori'].type_as(params_type)
        batch['forward']['eval_masks'] = data['indicating_mask'].type_as(params_type)

        batch['backward']['values'] = data['back_X'].type_as(params_type)
        batch['backward']['masks'] = data['back_missing_mask'].type_as(params_type)
        batch['backward']['deltas'] = data['back_deltas'].type_as(params_type)
        batch['backward']['evals'] = data['X_ori'].flip(1).type_as(params_type)
        batch['backward']['eval_masks'] = data['indicating_mask'].flip(1).type_as(params_type)

        batch['is_train'] = torch.ones(len(batch['forward']['values']))
        batch['labels'] = data['label'].long()

        batch = to_var(batch)
        
        ret_ric = model_ric.run_on_batch(batch, optimizer=optimizer_ric)